<a href="https://colab.research.google.com/github/shouvikcirca/corteva/blob/main/Data_Cleaning_Loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install sqlalchemy
# ! pip install python-dotenv

In [ ]:
import pandas as pd
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
! ls drive/MyDrive/Collabera

answers  README.md  src  wx_data  yld_data


In [ ]:
# ! git clone https://github.com/corteva/code-challenge-template.git drive/MyDrive/Collabera

Cloning into 'drive/MyDrive/Collabera'...
remote: Enumerating objects: 176, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 176 (delta 0), reused 0 (delta 0), pack-reused 172 (from 1)
Receiving objects: 100% (176/176), 9.51 MiB | 8.78 MiB/s, done.
Updating files: 100% (172/172), done.


In [ ]:
dataFiles = os.listdir('drive/MyDrive/Collabera/wx_data')
len(dataFiles)

167

In [ ]:
def parser(entry):
  finalList = []
  listEntry = []
  currentString = ''
  for c in entry:
    if c=='\t':
      listEntry.append(currentString)
      currentString = ''
    else:
      currentString+=c
  listEntry.append(currentString)

  listEntry = [int(str(i).strip()) for i in listEntry]
  return listEntry

In [ ]:
stationlist = []
datelist = []
maxtemplist = []
mintemplist = []
precipitationlist = []

counter = 0
for dF in dataFiles:
  counter+=1
  with open('drive/MyDrive/Collabera/wx_data/'+dF,'r') as f:
    print('------{}----'.format(counter))
    print(dF)
    rawData = f.read().split('\n')[:-1]
    parsedData = []
    for entry in rawData:
      try:
        parsedData.append(parser(entry))
      except Exception as e:
        print(e)
        print(entry)
        print('---')
    stationlist.extend([dF[:-4]]*len(parsedData))
    datelist.extend([str(entry[0])[:-4]+'-'+str(entry[0])[4:6]+'-'+str(entry[0])[6:] for entry in parsedData])
    maxtemplist.extend([entry[1] for entry in parsedData])
    mintemplist.extend([entry[2] for entry in parsedData])
    precipitationlist.extend([entry[3] for entry in parsedData])

In [ ]:
len(stationlist),len(datelist),len(maxtemplist),len(mintemplist),len(precipitationlist)

(1729957, 1729957, 1729957, 1729957, 1729957)

In [ ]:
dataFrame = pd.DataFrame({
    'Station':stationlist,
    'Date':datelist,
    'MaxTemp':maxtemplist,
    'MinTemp':mintemplist,
    'Precipitation':precipitationlist
    })

In [ ]:
dataFrame.head()

In [ ]:
a = load_dotenv('drive/MyDrive/.env')

In [ ]:
DATABASE_URL = os.environ['CORTEVA_DB_URL']

In [ ]:
engine = create_engine(DATABASE_URL)
dataFrame.to_sql("m_readings", engine, index=False, if_exists='replace')

957